In [3]:
!pip install pydantic fastapi uvicorn

# Exemplo utilizando Pydantic e FastAPI para fazer uma API de "to do list"

Será exibido a validação da data, geração de ID automático e se a tarefa está marcada como concluida

In [6]:
from uuid import uuid4, UUID
from datetime import date
from typing import List, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field, field_validator
from fastapi.testclient import TestClient

app = FastAPI()
todos: List["ToDoItem"] = []

class ToDoItem(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    title: str = Field(..., min_length=1)
    description: Optional[str] = None
    due_date: Optional[date] = None
    completed: bool = False

    @field_validator("due_date", mode="before")
    @classmethod
    def check_due_date_future(cls, v):
        if v is not None and v < date.today():
            raise ValueError("due_date não pode ser no passado")
        return v

@app.post("/todos", response_model=ToDoItem)
def create_todo(item: ToDoItem):
    todos.append(item)
    return item

@app.get("/todos", response_model=List[ToDoItem])
def list_todos():
    return todos

@app.patch("/todos/{item_id}/complete", response_model=ToDoItem)
def complete_todo(item_id: UUID):
    for todo in todos:
        if todo.id == item_id:
            todo.completed = True
            return todo
    raise HTTPException(status_code=404, detail="Tarefa não encontrada")

In [7]:
# Testando o TestClient
client = TestClient(app)

# 1) Criar uma tarefa
resp1 = client.post("/todos", json={"title": "Comprar leite"})
print("POST /todos:", resp1.status_code, resp1.json())

# 2) Listar tarefas
resp2 = client.get("/todos")
print("GET /todos:", resp2.status_code, resp2.json())

# 3) Marcar como concluída
tid = resp2.json()[0]["id"]
resp3 = client.patch(f"/todos/{tid}/complete")
print(f"PATCH /todos/{tid}/complete:", resp3.status_code, resp3.json())

POST /todos: 200 {'id': '4b9a4ad7-f1c5-443f-9012-da69aee51db9', 'title': 'Comprar leite', 'description': None, 'due_date': None, 'completed': False}
GET /todos: 200 [{'id': '4b9a4ad7-f1c5-443f-9012-da69aee51db9', 'title': 'Comprar leite', 'description': None, 'due_date': None, 'completed': False}]
PATCH /todos/4b9a4ad7-f1c5-443f-9012-da69aee51db9/complete: 200 {'id': '4b9a4ad7-f1c5-443f-9012-da69aee51db9', 'title': 'Comprar leite', 'description': None, 'due_date': None, 'completed': True}
